# xStream (row streaming) vs static results

In [5]:
import gzip
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import average_precision_score, roc_auc_score
import tqdm
from XStream_River import xStream

### Load datasets

In [4]:
data = gzip.open("data/cancer.gz", "r")

X, y = [], []

for i in data:
  i = (i.decode('utf-8')).split(",")
  i = [float(x) for x in i]
  X.append(np.array(i[:-1]))
  #X.append(i[:-1])
  y.append(i[-1])

X = np.array(X)
y = np.array(y)

In [45]:
#Row streaming results

#Check if same performance using pysad=
from pysad.models import xStream as xStream2
from pysad.evaluation.metrics import AUPRMetric, AUROCMetric

k = 100 
n_chains = 100
depth = 15
window_size = int(0.10*len(y))
# window_size = 25

metric = AUROCMetric()

for i in range(10): # Change the number of runs (depending if the dataset is big)

    #I think it does not make sense to shuffle the data
    # X_shuffled, y_shuffled = shuffle(X, y, random_state=i)
    
    model = xStream2(num_components=k, n_chains=n_chains, depth=depth, window_size=window_size) 

    all_scores = []
    for i, x in enumerate(tqdm.tqdm(X)):
       
        # anomalyscore = -model.predict_one(x)
        # anomalyscore = -model.learn_one(x)

        anomaly_score = model.score_partial(x)
        model = model.fit_partial(x)
        metric.update(y[i], anomaly_score)

        if i>=window_size:
            all_scores.append(anomaly_score[0])
        
        # model.learn_one(x)
        # model.fit_partial(x)
    
    print(f"auc is {metric.get()}.")
    

    # y_adjusted = y[window_size:]
    # print(average_precision_score(y_adjusted, all_scores))

    # chunks = [all_scores[j:j+window_size] for j in range(0, len(all_scores), window_size)]
    # y_chunks = [y_adjusted[j:j+window_size] for j in range(0, len(y_adjusted), window_size)]

    # AP_window = []
    # for i in range(len(y_chunks)-1):
    #     score = average_precision_score(y_chunks[i], chunks[i])
    #     AP_window.append(score)

    # OAP = average_precision_score(y_adjusted, all_scores) 
    # MAP = sum(AP_window)/len(AP_window)
    # AUC = roc_auc_score(y_adjusted, all_scores)

    # print("XStream: OAP =", OAP,"\n\t ",
    #     "MAP =", MAP, "\n\t ", 
    #     "AUC =", AUC)
    
    break

100%|██████████| 385/385 [00:20<00:00, 18.83it/s]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.